In [2]:
import pandas as pd
import os
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import SGDClassifier

In [3]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [46]:
pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5

In [2]:
train_path = r'C:\Users\tekienao\Desktop\preprocessed_dataset\train'
test_path = r'C:\Users\tekienao\Desktop\preprocessed_dataset\test'

# Function to load all CSV files from a directory into a single DataFrame
def load_data(directory):
    frames = []
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            file_path = os.path.join(directory, file)
            df = pd.read_csv(file_path)
            frames.append(df)
    return pd.concat(frames, ignore_index=True)

# Load training and testing data
train_data = load_data(train_path)
test_data = load_data(test_path)


In [5]:
train_data.columns

Index(['Y', 'Participant', 'EAR', 'MAR', 'PUC', 'MOE', 'Frame'], dtype='object')

In [6]:
train_data

,Y,Participant,EAR,MAR,PUC,MOE,Frame
0,0,1,0.319322,0.365214,0.537471,1.143717,0
1,0,1,0.287456,0.344043,0.513264,1.196854,1
2,0,1,0.294964,0.314391,0.516583,1.065861,2
3,0,1,0.285785,0.319771,0.511930,1.118919,3
4,0,1,0.289328,0.348412,0.515146,1.204208,4
...,...,...,...,...,...,...,...
2435114,5,48,0.246433,0.321256,0.469971,1.303625,18178
2435115,5,48,0.253011,0.315891,0.475356,1.248526,18179
2435116,5,48,0.258065,0.314562,0.476168,1.218923,18180
2435117,5,48,0.254269,0.324179,0.484533,1.274943,18181


In [10]:
print(train_data.isnull().sum())

Y              0
Participant    0
EAR            0
MAR            0
PUC            0
MOE            0
Frame          0
dtype: int64


In [12]:
train_data.dropna(subset=['EAR', 'MAR', 'PUC', 'MOE'], inplace=True)

In [13]:
print(train_data.isnull().sum())

Y              0
Participant    0
EAR            0
MAR            0
PUC            0
MOE            0
Frame          0
dtype: int64


In [14]:
train_data['Y']

0          0
1          0
2          0
3          0
4          0
          ..
2435114    5
2435115    5
2435116    5
2435117    5
2435118    5
Name: Y, Length: 2431600, dtype: int64

In [15]:
unique_values = train_data['Y'].value_counts()
print(unique_values)

Y
0     819991
10    812390
5     799219
Name: count, dtype: int64


In [16]:
print(test_data.isnull().sum())

Y              0
Participant    0
EAR            0
MAR            0
PUC            0
MOE            0
Frame          0
dtype: int64


In [17]:
test_data.dropna(subset=['EAR', 'MAR', 'PUC', 'MOE'], inplace=True)

In [18]:
print(test_data.isnull().sum())

Y              0
Participant    0
EAR            0
MAR            0
PUC            0
MOE            0
Frame          0
dtype: int64


In [19]:
X = train_data.drop('Y', axis=1)
y = train_data['Y']
XT = test_data.drop('Y', axis=1)
YT = test_data['Y']

In [20]:
le = LabelEncoder()
y = le.fit_transform(y)
YT = le.transform(YT)

In [17]:
model = xgb.XGBClassifier()
model.fit(X, y)

KeyboardInterrupt: 

In [43]:
# Perform cross-validation
cv_accuracy_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# Output cross-validation results
print(f'Cross-Validation Accuracy Scores: {cv_accuracy_scores}')
print(f'Mean CV Accuracy: {cv_accuracy_scores.mean():.2f}')
print(f'Standard Deviation of CV Accuracy: {cv_accuracy_scores.std():.2f}')

# Train the model on the entire training data
model.fit(X, y)

# Evaluate the model on the separate test set
y_pred = model.predict(XT)
test_accuracy = accuracy_score(YT, y_pred)
print(f"Test Set Accuracy: {test_accuracy:.2f}")

Cross-Validation Accuracy Scores: [0.37388345 0.37191767 0.32136865 0.31468169 0.33558768]
Mean CV Accuracy: 0.34
Standard Deviation of CV Accuracy: 0.02
Test Set Accuracy: 0.50


In [44]:
# Initialize the Random Forest classifier
model = RandomForestClassifier(random_state=42)

# Train the model
model.fit(X, y)

# Predict and evaluate
y_pred = model.predict(XT)
accuracy = accuracy_score(YT, y_pred)
print(f"Test Set Accuracy: {accuracy:.2f}")

Test Set Accuracy: 0.45


In [21]:
# Preprocessing the target
encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y.reshape(-1, 1))
YT_encoded = encoder.transform(YT.reshape(-1, 1))

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
XT_scaled = scaler.transform(XT)


C:\Users\tekienao\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [18]:
# Neural network architecture
model = Sequential()
model.add(Dense(128, input_shape=(X_scaled.shape[1],), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_encoded.shape[1], activation='softmax'))  # Output layer

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_scaled, y_encoded, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(XT_scaled, YT_encoded)
print(f"Test Accuracy: {test_accuracy:.2f}")



Epoch 1/10


60790/60790 [==============================] - 235s 4ms/step - loss: 0.7662 - accuracy: 0.6593 - val_loss: 2.0678 - val_accuracy: 0.3481
Epoch 2/10
60790/60790 [==============================] - 232s 4ms/step - loss: 0.6656 - accuracy: 0.7169 - val_loss: 2.5358 - val_accuracy: 0.3262
Epoch 3/10
60790/60790 [==============================] - 235s 4ms/step - loss: 0.6437 - accuracy: 0.7274 - val_loss: 3.0057 - val_accuracy: 0.3249
Epoch 4/10
60790/60790 [==============================] - 237s 4ms/step - loss: 0.6306 - accuracy: 0.7344 - val_loss: 2.8894 - val_accuracy: 0.3184
Epoch 5/10
60790/60790 [==============================] - 234s 4ms/step - loss: 0.6215 - accuracy: 0.7386 - val_loss: 3.3338 - val_accuracy: 0.3161
Epoch 6/10
60790/60790 [==============================] - 235s 4ms/step - loss: 0.6138 - accuracy: 0.7422 - val_loss: 3.0123 - val_accuracy: 0.3200
Epoch 7/10
60790/60790 [==============================] - 238s 4ms/step - loss: 0.6089 - accuracy: 0.7447 - 

In [ ]:
# Initialize the SVM model
# Note: The default kernel is 'rbf', but you can try others like 'linear', 'poly', etc.
svm_model = SVC(C=1.0, kernel='linear', random_state=42)

# Train the model
svm_model.fit(X_scaled, y)

# Predict and evaluate on the test set
y_pred = svm_model.predict(XT_scaled)
accuracy = accuracy_score(YT, y_pred)
print(f"Test Set Accuracy: {accuracy:.2f}")